## Import necessary modules
Run this cell before running any other cells

In [2]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

In [9]:
from uuid import uuid4
uuid4()

UUID('c330a0e0-98d2-4fd3-9f2c-0ab3bf3cac8a')

# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [2]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2024-01-23 16:18:32,490 | INFO     |: info
2024-01-23 16:18:32,492 | WARNING  |: warning
2024-01-23 16:18:32,493 | ERROR    |: error
2024-01-23 16:18:32,494 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [13]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2024-02-20 13:38:11,444 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:85:6a:10:3c
2024-02-20 13:38:14,086 | INFO     |: Connected to c0:83:85:6a:10:3c


## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [ ]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

In [7]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

T:74528


## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [47]:
ble.send_command(CMD.PING, "")

In [4]:
ble.send_command(CMD.ECHO, "the fitness gram pacer test")

In [9]:
ble.send_command(CMD.GET_TIME_MILLIS, "test")

2024-02-20 13:36:33,174 | INFO     |: 113406


In [15]:
ble.send_command(CMD.GET_MANY_TIME_MILLIS, "test")

2024-02-20 13:38:32,428 | INFO     |: 232803
2024-02-20 13:38:32,429 | INFO     |: 232803
2024-02-20 13:38:32,451 | INFO     |: 232803
2024-02-20 13:38:32,453 | INFO     |: 232803
2024-02-20 13:38:32,453 | INFO     |: 232823
2024-02-20 13:38:32,458 | INFO     |: 232833
2024-02-20 13:38:32,459 | INFO     |: 232841
2024-02-20 13:38:32,473 | INFO     |: 232852
2024-02-20 13:38:32,474 | INFO     |: 232852
2024-02-20 13:38:32,488 | INFO     |: 232852
2024-02-20 13:38:32,489 | INFO     |: 232872
2024-02-20 13:38:32,490 | INFO     |: 232872
2024-02-20 13:38:32,503 | INFO     |: 232882
2024-02-20 13:38:32,505 | INFO     |: 232882
2024-02-20 13:38:32,506 | INFO     |: 232892
2024-02-20 13:38:32,518 | INFO     |: 232901
2024-02-20 13:38:32,519 | INFO     |: 232901
2024-02-20 13:38:32,547 | INFO     |: 232912
2024-02-20 13:38:32,549 | INFO     |: 232912
2024-02-20 13:38:32,563 | INFO     |: 232912
2024-02-20 13:38:32,565 | INFO     |: 232932
2024-02-20 13:38:32,579 | INFO     |: 232932
2024-02-20

In [16]:
ble.send_command(CMD.SEND_TIME_DATA, "test")

2024-02-20 13:39:45,659 | INFO     |: 306123
2024-02-20 13:39:45,676 | INFO     |: 306123
2024-02-20 13:39:45,677 | INFO     |: 306123
2024-02-20 13:39:45,691 | INFO     |: 306123
2024-02-20 13:39:45,692 | INFO     |: 306123
2024-02-20 13:39:45,693 | INFO     |: 306123
2024-02-20 13:39:45,705 | INFO     |: 306123
2024-02-20 13:39:45,707 | INFO     |: 306123
2024-02-20 13:39:45,720 | INFO     |: 306123
2024-02-20 13:39:45,722 | INFO     |: 306123
2024-02-20 13:39:45,723 | INFO     |: 306123
2024-02-20 13:39:45,751 | INFO     |: 306123
2024-02-20 13:39:45,753 | INFO     |: 306123
2024-02-20 13:39:45,766 | INFO     |: 306123
2024-02-20 13:39:45,768 | INFO     |: 306123
2024-02-20 13:39:45,780 | INFO     |: 306123
2024-02-20 13:39:45,795 | INFO     |: 306123
2024-02-20 13:39:45,798 | INFO     |: 306123
2024-02-20 13:39:45,811 | INFO     |: 306123
2024-02-20 13:39:45,813 | INFO     |: 306123
2024-02-20 13:39:45,825 | INFO     |: 306123
2024-02-20 13:39:45,841 | INFO     |: 306123
2024-02-20

In [22]:
ble.send_command(CMD.GET_TEMP_READINGS, "test")

In [21]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

T:377903,514


The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [14]:
def timestamp_notification_handler(uuid, char_val_array):
    time = (int)(char_val_array.decode()[2:])
    LOG.info(time)

ble.start_notify(ble.uuid['RX_STRING'], timestamp_notification_handler)

In [19]:
stamped_temperature_data = []

def temperature_notification_handler(uuid, char_val_array):
    data = char_val_array.decode()[2:]
    timestamp, temp = data.split(',')
    timestamp = int(timestamp)
    temp = int(temp)
    stamped_temperature_data.append((timestamp, temp))
    
    # LOG.info(time)

ble.start_notify(ble.uuid['RX_STRING'], temperature_notification_handler)

In [23]:
ble.stop_notify(ble.uuid['RX_STRING'])

2024-02-20 13:44:30,173 | INFO     |: Disconnected from 324EB812-59E0-5CAF-B992-D25DCF474FBC


In [83]:
def print_temperature_data():
    LOG.info("temperature data: ")
    for timestamp, temp in stamped_temperature_data:
        LOG.info("time: " + str(timestamp) + ", temperature: " + str(temp))
    stamped_temperature_data.clear()

print_temperature_data()

2024-02-06 14:16:22,448 | INFO     |: temperature data: 
2024-02-06 14:16:22,449 | INFO     |: time: 411683, temperature: 509
2024-02-06 14:16:22,450 | INFO     |: time: 411683, temperature: 508
2024-02-06 14:16:22,450 | INFO     |: time: 411683, temperature: 509
2024-02-06 14:16:22,451 | INFO     |: time: 411683, temperature: 509
2024-02-06 14:16:22,452 | INFO     |: time: 411683, temperature: 509
2024-02-06 14:16:22,452 | INFO     |: time: 411683, temperature: 510
2024-02-06 14:16:22,453 | INFO     |: time: 411684, temperature: 509
2024-02-06 14:16:22,453 | INFO     |: time: 411684, temperature: 509
2024-02-06 14:16:22,454 | INFO     |: time: 411684, temperature: 509
2024-02-06 14:16:22,454 | INFO     |: time: 411684, temperature: 509
2024-02-06 14:16:22,454 | INFO     |: time: 411684, temperature: 510
2024-02-06 14:16:22,455 | INFO     |: time: 411684, temperature: 509
2024-02-06 14:16:22,455 | INFO     |: time: 411684, temperature: 510
2024-02-06 14:16:22,456 | INFO     |: time: 41

In [47]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

## Disconnect

In [12]:
# Disconnect
ble.disconnect()

2024-02-20 13:38:04,925 | INFO     |: Disconnected from 324EB812-59E0-5CAF-B992-D25DCF474FBC
